# Importing necessary libraries

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [33]:
#To ignore warnings

import warnings
warnings.filterwarnings("ignore")

In [34]:
pd.set_option('display.max_columns' , None)
pd.set_option('display.max_rows' , None)
pd.set_option('display.width' , None)

# Import Training Dataset

 In this example, We are considering that client has given us 2 files, train and test. In which testing data does not have predicted value
 
So we need to manually enter that value and submit it to client.
 
 Hence we will use training dataset entirely to train our model
 
 Now we will import training dataset to build a model and train it
 
 We have two options to check accuracy
 
 1- Create validation dataset
 
 2-K fold validation
 
 Since our dataset is small, if we create validation dataset our model will train on less number of datapoints
 
 Thus we will use K-fold validation 

In [38]:
train_data = pd.read_csv(r'C:\Users\Hp\Desktop\data science\Python\Datasets\risk_analytics_train.csv' , 
                           index_col = 0 , header = 0)
print(train_data.shape)
#print(test_data.shape)

(614, 12)


# EDA


In [37]:
#preprocessing the training data

train_data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [40]:
train_data.describe()

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,599.000000,614.000000,614.000000,592.000000,600.00000,564.000000
mean,0.762938,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,1.015216,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,0.000000,150.000000,0.000000,9.000000,12.00000,0.000000
25%,0.000000,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,0.000000,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,2.000000,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,3.000000,81000.000000,41667.000000,700.000000,480.00000,1.000000


# Missing value check

In [41]:
print(train_data.isnull().sum())                                   

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


# Missing value treatment

First we will impute missing value for categorical variable

In [6]:
train_data.dtypes

Gender                object
Married               object
Dependents           float64
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [42]:
colname=[]
for x in train_data.columns[:]:
    if train_data[x].dtype=='object':
        colname.append(x)
colname


['Gender',
 'Married',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [43]:
#filling will mode for categorical col

colname = ['Gender',
 'Married',
 'Dependents',
 'Self_Employed',
 'Loan_Amount_Term']

for x in colname:
    train_data[x].fillna(train_data[x].mode()[0] , inplace = True)

In [44]:
print(train_data.isnull().sum())      

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term      0
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


 We will impute Missing value of Loan Amount by its mean

In [45]:
#filling with mean for continous col

train_data['LoanAmount'].fillna(round(train_data['LoanAmount'].mean(), 0), inplace =True)
print(train_data.isnull().sum())

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


Credit history is an important variable, we are manually entering the value as per our domain knowledge

In [46]:
#manually entering the  value for important column as per domain knowledge

train_data['Credit_History'].fillna(value =0 , inplace=True)
     

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64


In [48]:
train_data.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# Coverting categorical to numerical variable

In [49]:
colname=[]
for x in train_data.columns[:]:
    if train_data[x].dtype=='object':
        colname.append(x)
colname


['Gender',
 'Married',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [50]:
from sklearn import preprocessing

colname = ['Gender',
 'Married',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

le = preprocessing.LabelEncoder()

for x in colname:
    train_data[x] = le.fit_transform(train_data[x])


In [51]:
train_data.head()


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,1,0,0.0,0,0,5849,0.0,146.0,360.0,1.0,2,1
LP001003,1,1,1.0,0,0,4583,1508.0,128.0,360.0,1.0,0,0
LP001005,1,1,0.0,0,1,3000,0.0,66.0,360.0,1.0,2,1
LP001006,1,1,0.0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
LP001008,1,0,0.0,0,0,6000,0.0,141.0,360.0,1.0,2,1


Target variable

Y--> 1

N --> 0

# Import testing dataset

In [55]:
test_data = pd.read_csv(r'C:\Users\Hp\Desktop\data science\Python\Datasets\risk_analytics_test.csv' , 
                              index_col = 0 , header = 0)

In [56]:
test_data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
Loan_ID,,,,,,,,,,,
LP001015,Male,Yes,0.0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
LP001022,Male,Yes,1.0,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
LP001031,Male,Yes,2.0,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
LP001035,Male,Yes,2.0,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
LP001051,Male,No,0.0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [57]:
print(train_data.shape)
print(test_data.shape)

(614, 12)
(367, 11)


Note - There is no Loan_status column in test dataset

# Missing value imputation

In [58]:
print(test_data.isnull().sum())                                   

Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64


In [59]:
#filling will mode for categorical/related col

colname = ['Gender',
 'Dependents',
 'Self_Employed',
 'Loan_Amount_Term']

for x in colname:
    test_data[x].fillna(test_data[x].mode()[0] , inplace = True)

In [60]:
test_data['LoanAmount'].fillna(round(test_data['LoanAmount'].mean(), 0), inplace =True)
print(test_data.isnull().sum())

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       29
Property_Area         0
dtype: int64


In [61]:
#manually entering the  value for important col as per domain knowledge

test_data['Credit_History'].fillna(value =0 , inplace=True)
print(test_data.isnull().sum()) 

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64


# Converting categorical to numerical

In [62]:
colname=[]
for x in test_data.columns[:]:
    if test_data[x].dtype=='object':
        colname.append(x)
colname

['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

In [63]:
from sklearn import preprocessing

colname = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

le = preprocessing.LabelEncoder()

for x in colname:
    test_data[x] = le.fit_transform(test_data[x])

In [64]:
test_data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
Loan_ID,,,,,,,,,,,
LP001015,1,1,0.0,0,0,5720,0,110.0,360.0,1.0,2
LP001022,1,1,1.0,0,0,3076,1500,126.0,360.0,1.0,2
LP001031,1,1,2.0,0,0,5000,1800,208.0,360.0,1.0,2
LP001035,1,1,2.0,0,0,2340,2546,100.0,360.0,0.0,2
LP001051,1,0,0.0,1,0,3276,0,78.0,360.0,1.0,2


# Train Test split

In [65]:
X_train = train_data.values[:,:-1]
Y_train = train_data.values[:,-1]
Y_train = Y_train.astype(int)

X_test = test_data.values[:,:]

# Scaling

In [66]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train) 

X_train = scaler.transform(X_train) 

X_test = scaler.transform(X_test)

# Model building and prediction

In [67]:
from sklearn import svm

svc_model = svm.SVC(kernel = 'rbf' , C=1.0 , gamma =0.1)
svc_model.fit(X_train , Y_train)
Y_pred = svc_model.predict(X_test)
print(list(Y_pred))

[1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 

We are importing dataset again, as we have converted few variables to numerical

For better understanding we will import dataset again and then add a new column called Y_predictions

In [69]:
test_data = pd.read_csv(r'C:\Users\Hp\Desktop\data science\Python\Datasets\risk_analytics_test.csv' , 
                           index_col = 0 , header = 0)
test_data['Y_predictions'] = Y_pred
test_data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Y_predictions
Loan_ID,,,,,,,,,,,,
LP001015,Male,Yes,0.0,Graduate,No,5720,0,110.0,360.0,1.0,Urban,1
LP001022,Male,Yes,1.0,Graduate,No,3076,1500,126.0,360.0,1.0,Urban,1
LP001031,Male,Yes,2.0,Graduate,No,5000,1800,208.0,360.0,1.0,Urban,1
LP001035,Male,Yes,2.0,Graduate,No,2340,2546,100.0,360.0,NaN,Urban,0
LP001051,Male,No,0.0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban,1


Copying our output in excel file to share it with client

In [70]:
test_data.to_csv('test_data.csv')

In [71]:
test_data.Y_predictions.value_counts()

1    285
0     82
Name: Y_predictions, dtype: int64

# K fold validation

In [75]:
#classifier = svm.SVC(kernel = 'rbf' , C=1.0 , gamma =0.1)
classifier = svm.SVC(kernel = 'rbf' , C=10.0 , gamma =0.001) #Tunning

#performing kfold_cross_validation
from sklearn.model_selection import KFold
kfold_cv=KFold(n_splits=10)
print(kfold_cv)

from sklearn.model_selection import cross_val_score
#running the model using scoring metric as accuracy
kfold_cv_result=cross_val_score(estimator=classifier,X=X_train, y=Y_train, cv=kfold_cv)
print(kfold_cv_result)
#finding the mean
print(kfold_cv_result.mean())



KFold(n_splits=10, random_state=None, shuffle=False)
[0.77419355 0.82258065 0.74193548 0.72580645 0.7704918  0.68852459
 0.80327869 0.7704918  0.7704918  0.83606557]
0.770386039132734
